In [1]:
import sys
sys.path.append("../")
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm

from const import gnps, mona
from utils import search_with_spectra, load_model, most_similar, cosine_similarity

In [2]:
show_progress_bar = False
model = load_model()

batch_size = None
k_metric = [5, 1, 10]
loader_batch_size = 512

replica_suffix = "-replication-{}"

In [3]:
spectra_paths = {
    "gnps":{
        "orbitrap": {
            "train": (gnps.ORBITRAP_TRAIN_QUERY, gnps.ORBITRAP_TEST_REF),
            "test": (gnps.ORBITRAP_TEST_QUERY, gnps.ORBITRAP_TEST_REF)
        },
        "qtof": {
            "test": (gnps.QTOF_TEST_QUERY, gnps.QTOF_TEST_REF)
        },
        "other": {
            "test": (gnps.OTHER_TEST_QUERY, gnps.OTHER_TEST_REF)
        }
    },
}

gnps_train_ref = np.load(gnps.ORBITRAP_TRAIN_REF, allow_pickle=True)

In [4]:
replica_df_seq = []
for i in tqdm(range(10)):
    df_seq = []
    for db, db_metadata in spectra_paths.items():
        for desc, path_metadata in db_metadata.items():
            for info, paths in path_metadata.items():
                print("-" * 40, f"{db}-{desc}-{info}", "-" * 40)
                query_path, ref_path = paths
                query_path = query_path.with_stem(query_path.stem + replica_suffix.format(i + 1))
                ref_path = ref_path.with_stem(ref_path.stem + replica_suffix.format(i + 1))
                if info == "train":
                    query_path = gnps.ORBITRAP_TRAIN_QUERY
                
                ref_spectra = np.load(ref_path, allow_pickle=True)
                query_spectra = np.load(query_path, allow_pickle=True)
                if desc == "orbitrap":
                    ref_spectra = np.hstack((gnps_train_ref, ref_spectra))
                
                df = search_with_spectra(
                    f"{db}-{desc}-{info}", model, 
                    query_spectra, ref_spectra,
                    k_metric, batch_size,
                    show_progress_bar
                )
                df_seq.append(df)
    df = pd.concat(df_seq, axis=0)
    print(df)
    replica_df_seq.append(df)

  0%|          | 0/10 [00:00<?, ?it/s]

---------------------------------------- gnps-orbitrap-train ----------------------------------------


6851it [00:11, 597.07it/s]
155415it [03:26, 751.57it/s]


---------------------------------------- gnps-orbitrap-test ----------------------------------------


1686it [00:03, 549.86it/s]
155415it [03:28, 746.77it/s]


...

---------------------------------------- gnps-qtof-test ----------------------------------------


7520it [00:05, 1300.57it/s]
37040it [00:32, 1152.05it/s]


---------------------------------------- gnps-other-test ----------------------------------------


6451it [00:06, 1051.58it/s]
44241it [00:40, 1084.50it/s]
100%|██████████| 10/10 [1:48:53<00:00, 653.31s/it]

                         top1      top5     top10
gnps-orbitrap-train  0.652168  0.798716  0.838272
gnps-orbitrap-test   0.625148  0.781139  0.823843
gnps-qtof-test       0.380718  0.544548  0.597074
gnps-other-test      0.502868  0.654627  0.704077


In [5]:
data = []
indices = replica_df_seq[0].index
columns = replica_df_seq[0].columns
for item in replica_df_seq:
    data.append([item.values])
data = np.concatenate(data, axis=0)
np.set_printoptions(precision=2, suppress=True)
np.mean(data, axis=0) * 100, np.std(data, axis=0) * 100

(array([[65.25, 79.87, 83.81],
        [63.23, 78.98, 83.25],
        [38.28, 55.07, 60.19],
        [50.66, 66.05, 70.85]]),
 array([[0.05, 0.05, 0.04],
        [0.77, 0.77, 0.64],
        [0.36, 0.31, 0.23],
        [0.52, 0.42, 0.46]]))

In [6]:
pd.set_option('display.precision', 2)
mean_df = pd.DataFrame(np.mean(data, axis=0) * 100, index=indices, columns=columns)
std_df = pd.DataFrame(np.std(data, axis=0) * 100, index=indices, columns=columns)
mean_df.to_csv("./mean.tsv", sep='\t')
std_df.to_csv("./std.tsv", sep='\t')

In [2]:
show_progress_bar = False
model = load_model()

batch_size = None
k_metric = [5, 1, 10]
loader_batch_size = 512

query_spectra = np.load("/data1/xp/data/MSBert/MTBLS1572/query.npy", allow_pickle=True)
ref_spectra = np.load("/data1/xp/data/MSBert/MTBLS1572/ref.npy", allow_pickle=True)

search_with_spectra(
    "MTBLS1572",
    model,
    query_spectra,
    ref_spectra,
    k_metric,
    batch_size,
    show_progress_bar
)

21it [00:00, 26.32it/s]
21it [00:00, 1319.28it/s]


,top1,top5,top10
MTBLS1572,0.809524,1.0,1.0


In [3]:
show_progress_bar = False
model = load_model()

batch_size = None
k_metric = [5, 1, 10]
loader_batch_size = 512

query_spectra = np.load("/data1/xp/data/MSBert/MTBLS1572/query.npy", allow_pickle=True)
ref_spectra = np.load("/data1/xp/data/MSBert/MTBLS1572/ref.npy", allow_pickle=True)

query_embedding = model.get_embedding_array(query_spectra)
ref_embedding = model.get_embedding_array(ref_spectra)

cosine_score = cosine_similarity(
    query_embedding, ref_embedding
)
for i, j in enumerate(np.argmax(cosine_score, axis=1)):
    if i != j:
        print(f"{i}-th answer is [{ref_spectra[i].get("compound_name")}] but get {j}-th [{ref_spectra[j].get("compound_name")}]")

21it [00:00, 1425.43it/s]
21it [00:00, 1495.04it/s]

4-th answer is [Butyryl-L-carnitine] but get 11-th [Isovaleryl-L-carnitine]
17-th answer is [Pantothenic acid] but get 19-th [Propionyl-L-carnitine]
18-th answer is [Proline] but get 7-th [Cytosine]
19-th answer is [Propionyl-L-carnitine] but get 11-th [Isovaleryl-L-carnitine]


In [4]:
cosine_score[4][4], cosine_score[4][11]

(0.7985360549130454, 0.966494467784608)

In [9]:
from utils import cosine_similarity

cosine_score = cosine_similarity(query_embedding, ref_embedding)

In [10]:
np.argsort(cosine_score, axis=1)[:, ::-1][:, 0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 16, 17, 20])

In [11]:
cosine_score[[18, 19], :]

array([[0.35, 0.31, 0.18, 0.2 , 0.19, 0.38, 0.48, 0.32, 0.11, 0.17, 0.56,
        0.2 , 0.67, 0.14, 0.15, 0.2 , 0.87, 0.58, 0.54, 0.31, 0.29],
       [0.3 , 0.48, 0.24, 0.32, 0.7 , 0.67, 0.59, 0.45, 0.17, 0.08, 0.68,
        0.72, 0.58, 0.18, 0.09, 0.16, 0.54, 0.98, 0.49, 0.85, 0.5 ]])

In [2]:
model = load_model()

orbitrap_ref_spectra = np.load(mona.ORBITRAP_COMMON, allow_pickle=True)
orbitrap_query_spectra = np.load(mona.ORBITRAP_UNIQUE, allow_pickle=True)

qtof_ref_spectra = np.load(mona.ORBITRAP_COMMON, allow_pickle=True)
qtof_query_spectra = np.load(mona.QTOF_UNIQUE, allow_pickle=True)

orbitrap_ref_embedding = model.get_embedding_array(orbitrap_ref_spectra)
orbitrap_query_embedding = model.get_embedding_array(orbitrap_query_spectra)

qtof_ref_embedding = model.get_embedding_array(qtof_ref_spectra)
qtof_query_embedding = model.get_embedding_array(qtof_query_spectra)

3376it [00:01, 1744.15it/s]
15655it [00:06, 2546.51it/s]
3376it [00:01, 2522.50it/s]
674it [00:00, 2528.02it/s]


In [3]:
orbitrap_score, orbitrap_indices = most_similar(orbitrap_query_embedding, orbitrap_ref_embedding, 512)
qtof_score, qtof_indices = most_similar(qtof_query_embedding, qtof_ref_embedding, 512)

processing: 100%|██████████| 2/2 [00:00<00:00, 25.84it/s]


In [4]:
dir = Path("/data1/xp/code/specEmbedding/score_distribution/compound search/")
orbitrap_dir = dir / "orbitrap"
qtof_dir = dir / "qtof"

orbitrap_dir.mkdir(exist_ok=True, parents=True)
qtof_dir.mkdir(exist_ok=True, parents=True)

np.save(orbitrap_dir / "MS2DeepScore_Score.npy", orbitrap_score)
np.save(orbitrap_dir / "MS2DeepScore_Indices.npy", orbitrap_indices)
np.save(qtof_dir / "MS2DeepScore_Score.npy", qtof_score)
np.save(qtof_dir / "MS2DeepScore_Indices.npy", qtof_indices)

In [5]:
show_progress_bar = False
model = load_model()

batch_size = None
k_metric = [5, 1, 10]
loader_batch_size = 512

In [6]:
query_spectra = np.load(mona.ORBITRAP_COMMON, allow_pickle=True)
ref_spectra = np.load(gnps.ORBITRAP_ALL, allow_pickle=True)

search_with_spectra(
    "Orbitrap Common",
    model,
    query_spectra,
    ref_spectra,
    k_metric,
    batch_size,
    show_progress_bar
)

3376it [00:01, 2039.29it/s]
163952it [01:09, 2374.03it/s]


,top1,top5,top10
Orbitrap Common,0.746149,0.784064,0.80628


In [7]:
query_spectra = np.load(mona.QTOF_COMMON, allow_pickle=True)
ref_spectra = np.load(gnps.QTOF_ALL, allow_pickle=True)

search_with_spectra(
    "QTOF Common",
    model,
    query_spectra,
    ref_spectra,
    k_metric,
    batch_size,
    show_progress_bar
)

7243it [00:03, 2408.12it/s]
44560it [00:19, 2236.87it/s]


,top1,top5,top10
QTOF Common,0.982052,0.993511,0.994754
